In [6]:
import pandas as pd
import nltk
import re
from itertools import groupby

In [20]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/morganedaniel/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [43]:
bdd_sbt_df = pd.read_csv("/Users/morganedaniel/Downloads/BDD_SBT_from_excel.csv")

In [44]:
bdd_sbt_df.head()


,Company Name,ISIN,Near term - Target Status,Near term - Target Classification,Near term - Target Year,Long term - Target Status,Long term - Target Classification,Long term - Target Year,Net-Zero Committed,Net-Zero Year,Organization Type,BA1.5?,BA1.5 Date,Country,Region,Sector,Date,Target,Target Classification,Extension
0,Hongkong Land Holdings Limited,BMG4587L1090,Committed,NaN,NaN,NaN,NaN,NaN,No,NaN,Company,No,NaN,Hong Kong,Asia,Real Estate,01/02/2022,NaN,NaN,NaN
1,Iress Limited,NaN,Committed,NaN,NaN,NaN,NaN,NaN,No,NaN,Company,No,NaN,Australia,Oceania,Software and Services,01/02/2022,NaN,NaN,NaN
2,Jacuzzi Brands,NaN,Committed,NaN,NaN,NaN,NaN,NaN,No,NaN,Company,No,NaN,United States of America (USA),North America,"Consumer Durables, Household and Personal Prod...",01/02/2022,NaN,NaN,NaN
3,Klimasan AŞ,NaN,Committed,NaN,NaN,NaN,NaN,NaN,No,NaN,Company,No,NaN,Turkey,Asia,"Consumer Durables, Household and Personal Prod...",01/02/2022,NaN,NaN,NaN
4,Mavi,TREMAVI00037,Committed,NaN,NaN,NaN,NaN,NaN,No,NaN,Company,No,NaN,Turkey,Asia,Retailing,01/02/2022,NaN,NaN,NaN


In [45]:
companies_with_target_df = bdd_sbt_df[bdd_sbt_df['Target'].notna()]

In [46]:
def refine_perimeter(perimeter_group):
    reg = r'([0-9]+.?[0-9]+?\ ?%) by ([0-9]+).*and'
    sub_group = re.search(reg, perimeter_group)
    if sub_group:
        return {"reduction_percentage": sub_group.groups()[0], "target_year": sub_group.groups()[1], "perimeter": re.sub(reg, "", perimeter_group) }
    

In [47]:
def which_scopes(scope_str):
    scope_tags = nltk.pos_tag(nltk.word_tokenize(scope_str))
    grammar =""" SCOPES: {<JJ>?<NN><CD><CC><VB>?<CD>}
                         {<JJ>?<NN><CD>}
            """
    cp = nltk.RegexpParser(grammar)
    result = cp.parse(scope_tags)
    if len(result) > 1:
        return
    for subtree in result.subtrees():
        if subtree.label() == 'SCOPES': 
            scopes = [token for token, tag in subtree.leaves() if tag == 'CD']
            return scopes
            

In [48]:
count = 0
target_info = []
target_info_not_found = [np.nan, np.nan, np.nan, np.nan, np.nan]

pattern = r'(reduce .*scope [0-9].* GHG emissions.* [0-9]+.?[0-9]+?\ ?% by [0-9]+ from a [0-9]+ base year)'

for index, row in  companies_with_target_df.iterrows():
    row_to_list = row.values.tolist()

    sentences = nltk.sent_tokenize(row['Target'])
    for sentence in sentences:
        results = re.findall(pattern, sentence)
        if not results:
            target_info.append(row_to_list+target_info_not_found)
            continue
        for result in results:
            results_groups = re.search(r'reduce (.*)(scope [0-9].*) (GHG emissions.*) ([0-9]+.?[0-9]+?\ ?%) by ([0-9]+) from a ([0-9]+) base year', result)
            if not results_groups:
                target_info.append(row_to_list+target_info_not_found)
                print(f"Clean Data at index {index},: {result}")
                continue
            reduction_percentages = [results_groups.groups()[3]]
            target_years = [results_groups.groups()[4]]
            base_year = results_groups.groups()[5]
            perimeter_group = results_groups.groups()[2]
    
            new_perimeter = refine_perimeter(perimeter_group)
            if perimeter_group == "GHG emissions" or not new_perimeter:
                perimeter = perimeter_group
            else:
                target_years.insert(0, new_perimeter["target_year"])
                reduction_percentages.insert(0, new_perimeter["reduction_percentage"])
                perimeter = new_perimeter["perimeter"]
            scopes = which_scopes(" ".join([results_groups.groups()[0], results_groups.groups()[1]]))
            if not scopes:
                target_info.append(row_to_list+target_info_not_found)
                continue
            for scope in scopes:
                for reduction_percentage, target_year in zip(reduction_percentages, target_years):
                    #print(f"scope: {scope}, reduction_percentage: {reduction_percentage}, target_year: {target_year}, base_year: {base_year}, perimeter: {perimeter}")
                    target_info.append(row_to_list+[scope, reduction_percentage, target_year, base_year, perimeter])

    

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [49]:
df_with_target_info = pd.DataFrame(target_info, columns = list(companies_with_target_df.columns)+["scope", "reduction_percentage", "target_year", "base_year", "perimeter"])

In [50]:
df_with_target_info.head()

,Company Name,ISIN,Near term - Target Status,Near term - Target Classification,Near term - Target Year,Long term - Target Status,Long term - Target Classification,Long term - Target Year,Net-Zero Committed,Net-Zero Year,...,Sector,Date,Target,Target Classification,Extension,scope,reduction_percentage,target_year,base_year,perimeter
0,(ACIP) Alexandria Company for Industrial Packages,NaN,Targets Set,Well-below 2°C,2030,NaN,NaN,NaN,No,NaN,...,Containers and Packaging,01/02/2022,This target was approved using a streamlined t...,The targets covering greenhouse gas emissions ...,NaN,NaN,NaN,NaN,NaN,NaN
1,(ACIP) Alexandria Company for Industrial Packages,NaN,Targets Set,Well-below 2°C,2030,NaN,NaN,NaN,No,NaN,...,Containers and Packaging,01/02/2022,This target was approved using a streamlined t...,The targets covering greenhouse gas emissions ...,NaN,1,25%,2030,2020,GHG emissions
2,(ACIP) Alexandria Company for Industrial Packages,NaN,Targets Set,Well-below 2°C,2030,NaN,NaN,NaN,No,NaN,...,Containers and Packaging,01/02/2022,This target was approved using a streamlined t...,The targets covering greenhouse gas emissions ...,NaN,2,25%,2030,2020,GHG emissions
3,AM Værktøj Odense A/S,NaN,Targets Set,1.5°C,2030,NaN,NaN,NaN,No,NaN,...,Technology Hardware and Equipment,01/02/2022,This target was approved using a streamlined t...,The targets covering greenhouse gas emissions ...,NaN,NaN,NaN,NaN,NaN,NaN
4,AM Værktøj Odense A/S,NaN,Targets Set,1.5°C,2030,NaN,NaN,NaN,No,NaN,...,Technology Hardware and Equipment,01/02/2022,This target was approved using a streamlined t...,The targets covering greenhouse gas emissions ...,NaN,1,50%,2030,2018,GHG emissions
